In [2]:
from tokenizer import fetch_messages, build_tokenizer, encode_messages

In [3]:
messages = fetch_messages()[:10000]
tokenizer = build_tokenizer(messages)
vocab_size = (tokenizer.get_vocab_size())
encoded = encode_messages(messages, tokenizer, None)

n = int(len(encoded)*.8)
train = encoded[:n]
test = encoded[n:]

saved tokenizer!!!


In [59]:
import torch 
device = 'cuda' if torch.cuda.is_available() else 'cpu'

block_size = 128
batch_size = 64
learning_rate = 3e-4
n_embd = 512
n_head = 8
dropout = 0.1
n_layer = 8

torch.manual_seed(42)

def get_batch(split):
  dataset = train if split == 'train' else test
  ixs = torch.randint(len(encoded)-(block_size+2), (batch_size,))
  x = torch.tensor([encoded[i:i+block_size] for i in ixs]) 
  y = torch.tensor([encoded[i+1:i+1+block_size] for i in ixs])

  x,y=x.to(device),y.to(device)

  return x,y

x_train,y_train = get_batch('train')
x_test,y_test = get_batch('test')

In [55]:
from torch import nn
import torch.nn.functional as F

class Head(nn.Module):
  
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)

    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):

    B,T,C = x.shape

    k = self.key(x)
    q = self.query(x)

    wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    v = self.value(x)
    out = wei @ v
    return out

class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(head_size * num_heads, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out

class FeedForward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
      nn.Linear(n_embd, 4*n_embd),
      nn.ReLU(),
      nn.Linear(4*n_embd, n_embd),
      nn.Dropout(dropout)
    )
  
  def forward(self, x):
    return self.net(x)

class Block(nn.Module):
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    y = self.sa(x)
    x = self.ln1(x+y)
    y = self.ffwd(x)
    x = self.ln2(x+y)
    return x 

class LanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])

    self.ln_f = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

    self.apply(self._init_weights)

  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
      torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
      if module.bias is not None:
        torch.nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
      torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
  
  def forward(self, index,targets=None):
    B, T  = index.shape
    
    tok_emb = self.token_embedding_table(index)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x)

    if targets is None:
      loss=None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits,loss
  
  def generate(self, index, max_new_tokens):
    for _ in range(max_new_tokens):
      #get predictions
      logits, loss = self.forward(index)

      logits = logits[:,-1,:]

      probs = F.softmax(logits, dim=-1)

      index_next = torch.multinomial(probs, num_samples=1)

      index = torch.cat((index, index_next), dim=1)

    return index

model = LanguageModel(vocab_size=vocab_size)
m = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

#context = torch.zeros((1,1),dtype=torch.long, device=device)
#generated_chars = tokenizer.decode((m.generate(context, max_new_tokens=300)[0].tolist()), skip_special_tokens=False)

In [56]:
max_iters = 10000
eval_iters = 100

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k]=loss.item()
    out[split] = losses.mean()

  model.train()
  return out 

In [66]:
import time

best_val_loss = float('inf')
best_model_state = None

checkpoint_path = "best_model.pth"
try:
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    print(f"Model loaded from '{checkpoint_path}'!")
except FileNotFoundError:
    print(f"No saved model found at '{checkpoint_path}'.")

times = []
for iter in range(max_iters):

  start = time.time()
  xb, yb = get_batch('train')

  logits, loss = m.forward(xb,yb)

  times.append(time.time()-start)
  
  if iter % eval_iters == 0:
    losses = estimate_loss()
    print(f"Step {iter} train loss: {losses['train']:.3f}, test loss: {losses['val']:.3f}, avg iter time: {sum(times)/len(times):.3f}")
    # Save the model if validation loss improves
    if losses['val'] < best_val_loss:
        best_val_loss = losses['val']
        best_model_state = m.state_dict()
        torch.save(best_model_state, checkpoint_path)
        print(f"saved w/ val loss: {best_val_loss:.3f}")


  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

Step 0 train loss: 2.290, test loss: 2.285, avg iter time: 0.069
saved w/ val loss: 2.285
Step 100 train loss: 2.260, test loss: 2.251, avg iter time: 0.255
saved w/ val loss: 2.251


KeyboardInterrupt: 

In [ ]:
output = tokenizer.encode("Essentially you just run the notebook on the remote ")

test = torch.tensor(data=[output.ids[:6]], device=device)

max_new_tokens = (block_size+1) - len(test[0])

if max_new_tokens > 0:
  print(tokenizer.decode(test[0].tolist()), end="")
  for i in range(100):
    generated_tok = m.generate(test, max_new_tokens=max_new_tokens)
    print(tokenizer.decode((generated_tok)[0][-max_new_tokens:].tolist(), skip_special_tokens=False), end="")
    test = generated_tok[:, -8:]
    max_new_tokens=1

essentially you just reward sharing reasons (work & slimeler, 2008).

self-trained plans, and passions also boasts maintaining pre-sale leaders allocated policies and processes while saving human errors undiliator unavel amidst authorier charges related to alterestate industry, fannide, roggery transcerront growth as some matrix equalized with x 1 and on that slowly important, consistent upon environmental livestinals. this doesn't enroled continued code that could capitalize your team, but also present in the curiofinity of 